In [1]:
## Basic import and DF creation

import pandas as pd

# Specify the path to your Excel file
file_path = r"09_Outputs\main_data.xlsx"

# Load the Excel file into a DataFrame using the openpyxl engine
df_main = pd.read_excel(file_path, engine='openpyxl')

# Remove quotation marks from the 'Entity' column
df_main['Entity'] = df_main['Entity'].str.replace('"', '')

# Retrieve unique countries from the DataFrame and store them as a list of strings
all_countries = df_main['Entity'].drop_duplicates().astype(str).tolist()

# Display the first 5 rows of the DataFrame for a quick overview
df_main.head()


,Entity,Year,Code,Annual CO₂ emissions in tons,Renewables - TWh growth - equivalent,Annual CO₂ emissions from flaring,Annual CO₂ emissions from gas,Annual CO₂ emissions from oil,Annual CO₂ emissions from coal,Primary energy consumption (TWh),...,Other renewables excluding bioenergy (TWh),Oil consumption - TWh,Primary energy consumption per GDP (kWh/$),Low-carbon electricity (TWh),GDP (constant 2015 US$),Renewable electricity (% electricity production) (World Bank (2015)),Solar photovoltaic capacity,Annual greenhouse gas emissions in CO2 equivalents,Continent,Electricity from wind (TWh)
0,Afghanistan,1965,AFG,1006917.0,NaN,0.0,0.0,542272.0,381056.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24430504.0,NaN,NaN
1,Afghanistan,1966,AFG,1091159.0,NaN,0.0,0.0,575248.0,428688.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25174948.0,NaN,NaN
2,Afghanistan,1967,AFG,1281865.0,NaN,0.0,260144.0,556928.0,399376.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24644464.0,NaN,NaN
3,Afghanistan,1968,AFG,1223391.0,NaN,0.0,347041.0,496817.0,332429.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24382198.0,NaN,NaN
4,Afghanistan,1969,AFG,941232.0,NaN,0.0,0.0,527616.0,362736.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23233260.0,NaN,NaN


In [4]:
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output
from IPython.core.display import HTML

# Assuming df_main is already available
filtered_df = df_main

# Extract unique countries from the dataframe
all_countries = sorted(filtered_df['Entity'].drop_duplicates().astype(str).tolist())
default_countries = ['Germany', 'China', 'India', 'United States', 'World']
left_container_width = '270px'

# Custom sorting function
def custom_sort(country):
    selected_countries = [cb.description for cb in checkboxes_container.children if cb.value]
    if country in selected_countries:
        return (-1, selected_countries.index(country))  # Selected countries come first
    else:
        return (1, country)  # All other countries come last

# Update list of countries based on user search input
def update_country_list(*args):
    selected_countries = [cb.description for cb in checkboxes_container.children if cb.value]
    search_input = search_box.value
    search_results = [country for country in all_countries if search_input.lower() in country.lower()]
    updated_countries = list(set(search_results + selected_countries))
    
    updated_countries = sorted(updated_countries, key=custom_sort)
    
    updated_checkboxes = [widgets.Checkbox(value=(country in selected_countries), description=country,
                                           layout=widgets.Layout(width="100%", padding="0px")) for country in updated_countries]
    checkboxes_container.children = tuple(updated_checkboxes)
    
    for cb in checkboxes_container.children:
        cb.observe(plot_selected_column, names='value')
        cb.observe(update_country_list, names='value') 

# The plot_selected_column function
def plot_selected_column(change=None):
    column = column_dropdown.value
    selected_countries = [cb.description for cb in checkboxes_container.children if cb.value]
    with out:
        clear_output(wait=True)
        fig = go.Figure()
        for country in selected_countries:
            country_data = filtered_df[filtered_df['Entity'] == country]
            fig.add_trace(go.Scatter(x=country_data['Year'], y=country_data[column], mode='lines', name=country, showlegend=True))
        
        # Added dotted grid layout
        fig.update_layout(title=column, 
                          xaxis=dict(title='Year', showgrid=True, gridcolor='rgba(128, 128, 128, 0.5)', gridwidth=0.5, griddash='dot', zeroline=False),
                          yaxis=dict(title=column, showgrid=True, gridcolor='rgba(128, 128, 128, 0.5)', gridwidth=0.5, griddash='dot', zeroline=False),
                          plot_bgcolor='white', paper_bgcolor='white')
        fig.show()


checkboxes_container = widgets.VBox([], layout=widgets.Layout(overflow_y="scroll", height="400px", border="1px solid #DDD"))

country_checkboxes = [widgets.Checkbox(value=(country in default_countries), description=country, layout=widgets.Layout(width="100%", padding="0px")) for country in sorted(all_countries, key=custom_sort)]

for cb in country_checkboxes:
    cb.observe(plot_selected_column, names='value')

checkboxes_container.children = tuple(country_checkboxes)

# Creating the search box widget
search_box = widgets.Text(value='', placeholder='Search for Countries...', description='', layout=widgets.Layout(width="95%"))
search_box.observe(update_country_list, names='value')

clear_button = widgets.Button(description="X", layout=widgets.Layout(width="5%"))
clear_button.on_click(lambda change: setattr(search_box, 'value', ''))

out = widgets.Output(layout=widgets.Layout(height="400px", width="60%"))

columns_to_plot = filtered_df.columns[filtered_df.columns.get_loc('Code')+1:]
column_dropdown = widgets.Dropdown(options=columns_to_plot, description='Select Column:')
column_dropdown.observe(plot_selected_column, names='value')

left_container = widgets.VBox([widgets.HBox([search_box, clear_button]), checkboxes_container], layout=widgets.Layout(width=left_container_width, border="1px solid #DDD", padding="5px 0px 0px 0px"))

display(HTML("""
<style>
    :root {
        --jp-widgets-inline-label-width: auto !important;
    }
</style>
"""))

display(column_dropdown)
display(widgets.HBox([left_container, out]))
plot_selected_column()
# Initial setup
update_country_list()

Dropdown(description='Select Column:', options=('Annual CO₂ emissions', 'Renewables (TWh growth - equivalent)'…

In [19]:
##################### Section 1: Imports and Data Assumptions #####################
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output
from IPython.core.display import HTML

# Assuming df_main is already loaded
filtered_df = df_main

##################### Section 2: Helper Functions #####################

# Extract unique countries from the dataframe
all_countries = sorted(filtered_df['Entity'].drop_duplicates().astype(str).tolist())
default_countries = ['Germany', 'China', 'India', 'United States', 'World']
left_container_width = '270px'

# Custom sorting function
def custom_sort(country):
    selected_countries = [cb.description for cb in checkboxes_container.children if cb.value]
    if country in selected_countries:
        return (-1, selected_countries.index(country))  # Selected countries come first
    else:
        return (1, country)  # All other countries come last

# Update list of countries based on user search input
def update_country_list(*args):
    selected_countries = [cb.description for cb in checkboxes_container.children if cb.value]
    search_input = search_box.value
    search_results = [country for country in all_countries if search_input.lower() in country.lower()]
    updated_countries = list(set(search_results + selected_countries))
    
    updated_countries = sorted(updated_countries, key=custom_sort)
    
    updated_checkboxes = [widgets.Checkbox(value=(country in selected_countries), description=country,
                                           layout=widgets.Layout(width="100%", padding="0px")) for country in updated_countries]
    checkboxes_container.children = tuple(updated_checkboxes)
    
    for cb in checkboxes_container.children:
        cb.observe(plot_selected_column, names='value')
        cb.observe(update_country_list, names='value') 

# The plot_selected_column function
def plot_selected_column(change=None):
    column = column_dropdown.value
    selected_countries = [cb.description for cb in checkboxes_container.children if cb.value]
    with out:
        clear_output(wait=True)
        fig = go.Figure()
        for country in selected_countries:
            country_data = filtered_df[filtered_df['Entity'] == country]
            fig.add_trace(go.Scatter(x=country_data['Year'], y=country_data[column], mode='lines', name=country, showlegend=True))
        
        # Added dotted grid layout
        fig.update_layout(title=column, 
                          xaxis=dict(title='Year', showgrid=True, gridcolor='rgba(128, 128, 128, 0.5)', gridwidth=0.5, griddash='dot', zeroline=False),
                          yaxis=dict(title=column, showgrid=True, gridcolor='rgba(128, 128, 128, 0.5)', gridwidth=0.5, griddash='dot', zeroline=False),
                          plot_bgcolor='white', paper_bgcolor='white')
        fig.show()


##################### Choropleth Map Function ##################### 
def plot_choropleth_map():
    with out:
        clear_output(wait=True)
        
        # Discretizing the data into 25 bins
        column_data = filtered_df[column_dropdown.value]
        bins = pd.cut(column_data, bins=25, labels=False, retbins=True)
        
        fig = go.Figure(data=go.Choropleth(
            locations=filtered_df['Code'],
            z=bins[0],  # Using the binned data here
            text=filtered_df['Entity'],
            colorscale='Viridis',
            autocolorscale=False,
            reversescale=True,
            marker_line_color='darkgray',
            marker_line_width=0.5,
            colorbar_title=column_dropdown.value,
            colorbar_tickvals=list(range(25)),
            colorbar_ticktext=[f'{round(bins[1][i], 2)}-{round(bins[1][i+1], 2)}' for i in range(25)]
        ))
        fig.update_layout(title_text='Choropleth Map of ' + column_dropdown.value,
                          geo=dict(showframe=False, showcoastlines=False, projection_type='equirectangular'))
        fig.show()



##################### Section 3: Create Widgets #####################
all_countries = sorted(filtered_df['Entity'].drop_duplicates().astype(str).tolist())
default_countries = ['Germany', 'China', 'India', 'United States', 'World']

checkboxes_container = widgets.VBox([], layout=widgets.Layout(overflow_y="scroll", height="400px", border="1px solid #DDD"))

country_checkboxes = [widgets.Checkbox(value=(country in default_countries), description=country, layout=widgets.Layout(width="100%", padding="0px")) for country in sorted(all_countries, key=custom_sort)]

for cb in country_checkboxes:
    cb.observe(plot_selected_column, names='value')

checkboxes_container.children = tuple(country_checkboxes)

# Creating the search box widget
search_box = widgets.Text(value='', placeholder='Search for Countries...', description='', layout=widgets.Layout(width="95%"))
search_box.observe(update_country_list, names='value')

clear_button = widgets.Button(description="X", layout=widgets.Layout(width="5%"))
clear_button.on_click(lambda change: setattr(search_box, 'value', ''))

out = widgets.Output(layout=widgets.Layout(width="60%"))


columns_to_plot = filtered_df.columns[filtered_df.columns.get_loc('Code')+1:]
column_dropdown = widgets.Dropdown(options=columns_to_plot, description='Select Column:')
column_dropdown.observe(plot_selected_column, names='value')

left_container = widgets.VBox([widgets.HBox([search_box, clear_button]), checkboxes_container], layout=widgets.Layout(width=left_container_width, border="1px solid #DDD", padding="5px 0px 0px 0px"))

##################### Toggle Button for Chart and Map ##################### 
view_toggle = widgets.ToggleButtons(
    options=['Chart', 'Map'],
    description='View:',
    value='Chart'
)
def toggle_view(change):
    if view_toggle.value == 'Chart':
        plot_selected_column()
    else:
        plot_choropleth_map()
view_toggle.observe(toggle_view, names='value')

##################### Display and Initialize Widgets #####################
display(HTML("""
<style>
    :root {
        --jp-widgets-inline-label-width: auto !important;
    }
</style>
"""))

display(view_toggle)
display(column_dropdown)
display(widgets.HBox([left_container, out]))
toggle_view(None)  # Initialize the view based on the initial value of the toggle button



ToggleButtons(description='View:', options=('Chart', 'Map'), value='Chart')

Dropdown(description='Select Column:', options=('Annual CO₂ emissions in tons', 'Renewables - TWh growth - equ…

In [18]:
##################### Section 1: Imports and Data Assumptions #####################

# Import necessary libraries
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output
from IPython.core.display import HTML
import plotly.express as px

# Assuming df_main is already loaded
filtered_df = df_main

##################### Section 2: Helper Functions #####################

# Extract unique countries from the dataframe
all_countries = sorted(filtered_df['Entity'].drop_duplicates().astype(str).tolist())
default_countries = ['Germany', 'China', 'India', 'United States', 'World']
left_container_width = '270px'

# Custom sorting function
def custom_sort(country):
    selected_countries = [cb.description for cb in checkboxes_container.children if cb.value]
    if country in default_countries:
        return (-2, default_countries.index(country))  # Default countries come first
    elif country in selected_countries:
        return (-1, selected_countries.index(country))  # Selected countries come second
    else:
        return (1, country)  # All other countries come last

# Update list of countries based on user search input
def update_country_list(*args):
    selected_countries = [cb.description for cb in checkboxes_container.children if cb.value]
    search_input = search_box.value
    search_results = [country for country in all_countries if search_input.lower() in country.lower()]
    updated_countries = list(set(search_results + selected_countries))
    
    updated_countries = sorted(updated_countries, key=custom_sort)
    
    updated_checkboxes = [widgets.Checkbox(value=(country in selected_countries), description=country,
                                           layout=widgets.Layout(width="100%", padding="0px")) for country in updated_countries]
    checkboxes_container.children = tuple(updated_checkboxes)
    
    for cb in checkboxes_container.children:
        cb.observe(plot_selected_column, names='value')
        cb.observe(update_country_list, names='value') 

# The plot_selected_column function
def plot_selected_column(change=None):
    column = column_dropdown.value
    selected_countries = [cb.description for cb in checkboxes_container.children if cb.value]
    
    # Load data for selected countries and column
    country_data = filtered_df[filtered_df['Entity'].isin(selected_countries)][['Year', column, 'Entity']]
    
    with out:
        clear_output(wait=True)
        fig = go.Figure()
        
        # Add line trace for each selected country
        for country in selected_countries:
            country_trace = country_data[country_data['Entity'] == country]
            fig.add_trace(go.Scatter(x=country_trace['Year'], y=country_trace[column], name=country))
        
        # Add scatter trace for all selected countries
        fig.add_trace(go.Scatter(x=country_data['Year'], y=country_data[column], mode='markers', name='All Selected Countries'))
        
        # Set layout and show plot
        fig.update_layout(title=column, 
                          xaxis=dict(title='Year', showgrid=True, gridcolor='rgba(128, 128, 128, 0.5)', gridwidth=0.5, griddash='dot', zeroline=False),
                          yaxis=dict(title=column, showgrid=True, gridcolor='rgba(128, 128, 128, 0.5)', gridwidth=0.5, griddash='dot', zeroline=False),
                          plot_bgcolor='white', paper_bgcolor='white')
        fig.show()

##################### Section 3: Create Widgets #####################

# Define initial values for widgets
all_countries = sorted(filtered_df['Entity'].drop_duplicates().astype(str).tolist())
default_countries = ['Germany', 'China', 'India', 'United States', 'World']

# Create checkboxes for countries
checkboxes_container = widgets.VBox([], layout=widgets.Layout(overflow_y="scroll", height="400px", border="1px solid #DDD"))
country_checkboxes = [widgets.Checkbox(value=(country in default_countries), description=country, layout=widgets.Layout(width="100%", padding="0px")) for country in sorted(all_countries, key=custom_sort)]

# Add observers to checkboxes
for cb in country_checkboxes:
    cb.observe(plot_selected_column, names='value')

# Add checkboxes to container
checkboxes_container.children = tuple(country_checkboxes)

# Create search box widget
search_box = widgets.Text(value='', placeholder='Search for Countries...', description='', layout=widgets.Layout(width="95%"))
search_box.observe(update_country_list, names='value')

# Create clear button for search box
clear_button = widgets.Button(description="X", layout=widgets.Layout(width="5%"))
clear_button.on_click(lambda change: setattr(search_box, 'value', ''))

# Create output widget for plot
out = widgets.Output(layout=widgets.Layout(height="600px", width="800px"))

# Create dropdown for selecting column to plot
columns_to_plot = filtered_df.columns[filtered_df.columns.get_loc('Code')+1:]
column_dropdown = widgets.Dropdown(options=columns_to_plot, description='Select Column:')
column_dropdown.observe(plot_selected_column, names='value')

# Create left container for search box and checkboxes
left_container = widgets.VBox([widgets.HBox([search_box, clear_button]), checkboxes_container], layout=widgets.Layout(width=left_container_width, border="1px solid #DDD", padding="5px 0px 0px 0px"))

##################### Section 3.1: Active Selection #####################

# Define a function to get the list of selected countries
def get_selected_countries():
    selected_countries = [cb.description for cb in checkboxes_container.children if cb.value]
    return selected_countries

# Define a function to update the list of selected countries
def update_selected_countries(selected_countries):
    for cb in checkboxes_container.children:
        if cb.description in selected_countries:
            cb.value = True
        else:
            cb.value = False

# Get the initial list of selected countries
selected_countries = get_selected_countries()

# Update the list of selected countries when a new country is selected
def update_active_selection(change):
    global selected_countries
    new_selected_countries = get_selected_countries()
    if new_selected_countries != selected_countries:
        selected_countries = new_selected_countries
        update_selected_countries(selected_countries)
        plot_selected_column()

# Add an observer to the checkboxes container to update the active selection
for cb in checkboxes_container.children:
    cb.observe(update_active_selection, names='value')

##################### Section 4: Toggle Button for Chart and Map ##################### 

# Create toggle button for switching between chart and map view
view_toggle = widgets.ToggleButtons(
    options=['Chart', 'Map'],
    description='View:',
    value='Chart'
)

##################### Section 5: Choropleth Map Function ##################### 

# The plot_choropleth_map function
def plot_choropleth_map():
    column = column_dropdown.value
    selected_years = sorted(filtered_df['Year'].drop_duplicates().astype(str).tolist())
    column_data = filtered_df[['Year', column, 'Code', 'Entity']]

    # Discretize the data into 8 bins
    bins = pd.cut(column_data[column_dropdown.value], bins=8, labels=False, retbins=True)
    bin_labels = [f'{round(bins[1][i], 2)}-{round(bins[1][i+1], 2)}' for i in range(8)]

    # Create the choropleth map using plotly.graph_objects
    fig = go.Figure(go.Choropleth(locations=column_data['Code'], z=bins[0],
                                   text=column_data['Entity'], colorscale='Plasma_r',
                                   zmin=0, zmax=7, colorbar=dict(title=column, tickvals=list(range(8)),
                                                                 ticktext=bin_labels)))

    fig.update_layout(title_text='Choropleth Map of ' + column_dropdown.value,
                      geo=dict(showframe=False, showcoastlines=False, projection_type='equirectangular'),
                      width=800, height=600)
    fig.show()

##################### Section 6: Set up observers ##################### 

# Add observer to toggle button
def toggle_view(change):
    if view_toggle.value == 'Chart':
        plot_selected_column()
    else:
        plot_choropleth_map()

view_toggle.observe(toggle_view, names='value')

##################### Section 7: Display and Initialize Widgets ##################### 

# Create button for plotting choropleth map
map_button = widgets.Button(description='Map')

# Add observer to map button
def plot_map_on_click(button):
    plot_choropleth_map()

map_button.on_click(plot_map_on_click)

# Display and initialize widgets
display(HTML("""
<style>
    :root {
        --jp-widgets-inline-label-width: auto !important;
    }
    .jp-CodeCell.jp-mod-outputsScrolled .jp-Cell-outputArea {
        max-height: none;
    }
</style>
"""))

display(view_toggle)
display(column_dropdown)
display(widgets.HBox([left_container, out]))
display(map_button)  # Display the map button
toggle_view(None)  # Initialize the view based on the initial value of the toggle button

ToggleButtons(description='View:', options=('Chart', 'Map'), value='Chart')

Dropdown(description='Select Column:', options=('Annual CO₂ emissions in tons', 'Renewables - TWh growth - equ…

Button(description='Map', style=ButtonStyle())